# Group Assignment 3 - UP221 Winter 2024
## Group Name: Food Access in LA County
### Madi Hamilton, Jessica Fay, Meaghan Woody, Branden Bohrnsen

Research Question - are there geographic disparities trends in food insecurity and coronary heart disease in Los Angeles County? 

Updates: new dataset to include SPAs, regraphed the data for the poverty and predictor/outcome variables with new dataset, included function and loop for fast food data

# Part 1 - Load Data
Data sources: USC Neighborhood Data for Social Change data, ACS census data

In [12]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import contextily as cx
import folium
import plotly.io as pio
import plotly.express as px

In [13]:
# import new dataset for USC data merged with SPAs
new = pd.read_csv('Data/new.csv', dtype= {'geoid20_x':str})
new.head(3)
new.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89453 entries, 0 to 89452
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SPA22              89399 non-null  float64
 1   SPA_NAME           89399 non-null  object 
 2   geoid20_x          89453 non-null  object 
 3   pop_below_100_pct  89302 non-null  float64
 4   pop_below_200_pct  89302 non-null  float64
 5   chd_pct            89453 non-null  float64
 6   total_pop          72154 non-null  float64
 7   snap_rate          72137 non-null  float64
 8   FIPS               89453 non-null  int64  
dtypes: float64(6), int64(1), object(2)
memory usage: 6.1+ MB


In [26]:
# import census tract data to merge data with tract data in next step
tracts = gpd.read_file('Data/tl_2020_06_tract.shp')
tracts.head(4)
tracts.info(verbose=True,show_counts=True)

tracts['geoid20_x']=tracts.TRACTCE
tracts.plot(figsize=(12,10))

tracts['COUNTYFP'].unique()

#filter for LA County census tracts
latracts = tracts.query("COUNTYFP == '037'")
latracts.head(5)
latracts.plot(figsize=(12,10))

DriverError: Unable to open Data/tl_2020_06_tract.shx or Data/tl_2020_06_tract.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.

In [19]:
#merge the data
tracts_census= latracts.merge(new,on="geoid20_x")

#check map of census tracts merged with usc/spa data
latracts.plot(figsize=(12,10))

#make sure is a geodataframe
tracts_census.info(verbose=True,show_counts=True)
tracts_census.head(5)

NameError: name 'latracts' is not defined

# Part 2 - Descriptive statistics

In [ ]:
#look at the mean and median population
tracts_census['total_pop'].mean() #mean per census tract
tracts_census['total_pop'].median() #median across census tracts

#look at mean and median snap acceptance rate
tracts_census['snap_rate'].mean() #mean per census tract
tracts_census['snap_rate'].median() #median across census tracts

#box plot
tracts_census.boxplot(column=['snap_rate'])

In [ ]:
# make boxplots for the distribution of key variables
df = pd.read_csv('Data/new.csv')

#SNAP Rate
px.box(data_frame = df
       ,x = 'snap_rate', 
       labels ={
                "snap_rate": "Grocery Store SNAP Acceptance Rate"},
       
       title='Census Tract Grocery Store SNAP Acceptance Rate'
       )
# Coronary Heart Disease Percent
px.box(data_frame = df
       ,x = 'chd_pct', 
        labels ={
                "chd_pct": "CHD Rate"},
       title='Census Tract Prevalence of Coronary Heart Disease'
       )

In [ ]:
# Look at distribution of Hispanic because in LA, most SNAP participants are Hispanic
px.box(data_frame = df
       ,x = 'pop_below_100_pct', title='Census Tract Percent Hispanic/Latino'
       )

In [ ]:
# Look at distribution of % of census that are 100% below poverty threshold
px.box(data_frame = df
       ,x = 'pop_below_200_pct', 
       labels ={
                "pop_below_100_pct": "Percent of Pop Below 100% Poverty Threshold",},
       title='Census Tract Percent Below 100% of Poverty Threshold'
       )

In [ ]:
# Initial Correlation of SNAP acceptance and CHD
 px.scatter(df, x="snap_rate", y="chd_pct",
            labels ={
                "chd_pct": "CHD Rate",
                "snap_rate": "Grocery Store SNAP Acceptance Rate"},
            title="Census Tract Coronary Heart Disease Rate vs. Grocery Store SNAP Acceptance Rate")


# Part 3 - Loops using OSM (Jessica)

In [ ]:
def chd_by_spa (place):
    fig, ax = plt.subplots(figsize=(10,10))
    # create the map plot
fig, ax = plt.subplots(figsize=(10,10))
tracts_census.plot(ax=ax,
         column='snap_rate',
         cmap='tab20',
         legend=True)

In [ ]:
# Define aread of interests
place = 'Downtown, Los Angeles, CA'

osm = ox.features.features_from_address(place, tags={'amenity': ['restaurant','fast_food']})

# examine data
osm.shape
osm.info(verbose=True, show_counts=True)

# subset data set to keep only variables of interest
columns_to_keep = ['amenity','name','brand', 'diet:vegan', 'access', 'geometry']
osm = osm[columns_to_keep]
osm.head(10)

In [ ]:
# get the counts of outlet by restaurant type
osm_food_counts = osm.value_counts(['amenity']).reset_index(name="count")
osm_food_counts

# create bar chart of food outlet type
osm_food_counts.plot.bar(x='amenity');

In [ ]:
# create enhanced bar chart

# create empty figure and axis where dataframe will be plotted
fig, ax = plt.subplots(figsize=(12,4))

# plot top ten building types with counts 
osm_food_counts[:10].plot.bar(ax=ax,
                                      x='amenity',
                                      y='count',
                                      legend=False,
                                      color='dodgerblue'
                                      )

ax.set_xlabel('Food Outlet Type') # override x label
ax.set_ylabel('Number of Outlets') # override y label
ax.set_title("Food outlet types\n"+place,fontsize=14,pad=10); # multi-line title with padding

In [ ]:
# list the templates available
pio.templates

In [ ]:
# set template theme
pio.templates.default = "seaborn"

In [ ]:
# make bar chart with new theme
fig = px.bar(osm_food_counts.head(10),
        x='amenity',
        y='count',
        title="food outlet types in "+place, # title
        text_auto = True,
        height=600,
        width=900,
        color_discrete_sequence =['slategray']*len(osm_food_counts), # single color for all the bars
        labels={
                'count': 'Number of outlets',
                'amenity': 'Type of food outlet',
        })

# bar label
fig.update_traces(textposition='outside',textfont_size=10,textfont_color='#444')

# axes labels
fig.update_yaxes(title_font_size=12,title_font_color='#aaa',tickfont_color='#aaa',tickfont_size=9)
fig.update_xaxes(title_font_size=10,title_font_color='#aaa',tickfont_color='#aaa',tickfont_size=9)

# show the figure
fig.show()
plt.savefig('city-name.png')

In [ ]:
#examine osm data again
osm.shape
osm.dtypes

#create a plot of the fast food and restaurants in dtla
osm.plot(figsize=(10,10),
         column='amenity',
         cmap='tab20',
         legend=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# create the map plot
osm.plot(ax=ax,
         column='amenity',
         cmap='tab20',
         legend=True,
         legend_kwds={'loc':'upper left','bbox_to_anchor':(1,.9)})

# add a title
ax.set_title('Food Outlets types in ' + place)

# get rid of the axis
ax.axis('off');

In [ ]:
# define geography bounds
minx = osm.total_bounds[0]
miny = osm.total_bounds[1]
maxx = osm.total_bounds[2]
maxy = osm.total_bounds[3]

# get unique buiding types in a list
foodtypes = osm['amenity'].unique().tolist()
foodtypes

In [ ]:
# loop through food outlet types
for type in foodtypes:
    fig, ax = plt.subplots(figsize=(4,4))

    # create the map plot
    osm.plot(ax=ax,
            # column='amenity',
            color='#eee')

    # create the map plot
    osm[osm['amenity'] == type].plot(ax=ax,
            # column='amenity',
            color='black')

    # set the extent of the map 
    # so that each map has the same bounds
    ax.set_xlim((minx,maxx))
    ax.set_ylim((miny,maxy))

    # add a title
    number_of_outlets = len(osm[osm['amenity']==type])
    ax.set_title(str(number_of_outlets) + ' ' + type + 'amenity')

    # get rid of the axis
    ax.axis('off');

In [ ]:
 # add a base map
#reproject to Web Mercator
osm_web_mercator = osm.to_crs(epsg=3857)

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))

osm_web_mercator.plot(ax=ax,
                    color="black",
                    alpha=0.8,
                    )

# get rid of the axis
ax.axis('off');

# basemap from carto that has a dark background (easier to see)
ctx.add_basemap(ax=ax,
                source=ctx.providers.CartoDB.Positron,
                alpha=0.3 # add transparency to make it less dominant
                )
plt.savefig('LA.png')

In [ ]:
# create a function to create a map
def make_food_map(place):
 
    # get the data from osm
    osm = ox.features.features_from_address(place,
                                    tags={'amenity': ['restaurant','fast_food']},
                                    dist=500)

    # reproject to Web Mercator
    osm_web_mercator = osm.to_crs(epsg=3857)

    # create the figure as a subplot
    fig,ax = plt.subplots(figsize=(10,10))
    
    # add the map
    osm_web_mercator.plot(ax=ax,
                          column='amenity',
                          cmap='tab20',
                          legend=True,
                          legend_kwds={'loc' :'upper left','bbox_to_anchor':(1,1)})
                
    # add a title
    ax.set_title("Food outlet Types in" + place)

    # get rid of the axis
    ax.axis('off')  
    
    #add a dark basemap
    ctx.add_basemap(ax,source=ctx.providers.CartoDB.DarkMatter)

In [ ]:
#make a list of places to loop through
place_list=[' Downtown, Los Angeles, CA', 'Westlake, Los Angeles', 'Westwood, Los Angeles, CA']

In [ ]:
#execute for loop
for place in place_list:
    make_food_map(place)

# Part 4 - Choropleth Maps (Madi, Meaghan)

In [ ]:
ax = tracts_census.plot(figsize=(12,10),
                   column='pop_below_100_pct',
                   legend=True,
                   scheme='NaturalBreaks',
                    cmap='YlOrRd')
ax.set_axis_off()
ax.set_title('Percent of the Population with Income Below 100% Poverty Level from 2016-2020')
cx.add_basemap(ax,crs=4326,
               source=cx.providers.CartoDB.Positron)
plt.savefig('pop_below_100_pct.png')

In [ ]:
ax = tracts_census.plot(figsize=(12,10),
                   column='pop_below_200_pct',
                   legend=True,
                   scheme='NaturalBreaks',
                    cmap='YlOrRd')
ax.set_axis_off()
ax.set_title('Percent of the Population with Income Below 100% Poverty Level from 2016-2020')
cx.add_basemap(ax,crs=4326,
               source=cx.providers.CartoDB.Positron)
plt.savefig('pop_below_100_pct.png')

In [ ]:
ax = tracts_census.plot(figsize=(12,10),
                   column='chd_pct',
                   legend=True,
                   scheme='NaturalBreaks',
                    cmap='GnBu')
ax.set_axis_off()
ax.set_title('Percent of the Population with Coronary Heart Disease in 2020')
cx.add_basemap(ax,crs=4326,
               source=cx.providers.CartoDB.Positron)
plt.savefig('chd_pct.png')

In [ ]:
ax = tracts_census.plot(figsize=(12,10),
                   column='snap_rate',
                   legend=True,
                   scheme='NaturalBreaks',
                    cmap='YlGnBu')
ax.set_axis_off()
ax.set_title('Number of SNAP Accepting Institutions per 10,000 Residents in a Census Tract in 2023')
cx.add_basemap(ax,crs=4326,
               source=cx.providers.CartoDB.Positron)
plt.savefig('snap_rate.png')

# Part 5 - Folium Maps (Branden)

In [ ]:
# Coerce to GDF, extract longitude and latitude from geometry column

df = pd.read_excel("output_JF.xlsx")

df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.set_crs(epsg=4326, inplace=True)

gdf_projected = gdf.to_crs(epsg=2163)
gdf_projected['centroid'] = gdf_projected.geometry.centroid
centroids_geo = gdf_projected['centroid'].to_crs(epsg=4326)

gdf['longitude'] = centroids_geo.x
gdf['latitude'] = centroids_geo.y

In [ ]:
gdf.head()
gdf.columns
gdf.snap_rate

In [ ]:
# Get proper data types
df['snap_rate'] = gdf['snap_rate'].astype(float)
gdf['chd_pct'] = gdf['chd_pct'].astype(float)

In [ ]:
# Create map with snap_rate as the fill variable

snap_rate_min = gdf['snap_rate'].min()
snap_rate_max = gdf['snap_rate'].max()

colormap = cm.linear.YlOrRd_09.scale(snap_rate_min, snap_rate_max)
colormap.caption = 'SNAP Rate'

def style_function(feature):
    snap_rate = feature['properties']['snap_rate']
    return {
        'fillColor': colormap(snap_rate),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

m = folium.Map(location=[gdf['latitude'].mean(), gdf['longitude'].mean()], zoom_start=5)

folium.GeoJson(
    gdf,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['snap_rate', 'chd_pct'], 
                                  aliases=['SNAP Rate:', 'CHD %:'], 
                                  localize=True),
    popup=folium.GeoJsonPopup(fields=['snap_rate', 'chd_pct'], 
                              aliases=['SNAP Rate:', 'CHD %:'], 
                              labels=True)
).add_to(m)

colormap.add_to(m)

m

# Group Member Contributions

1. Branden - folium
2. Jessica - cleaned data for import and analysis, conducted data exploration and developed visualizations of key variables
3. Madi - I created three chorpleth maps describing the spatial distribution of the percent of the population below the 100% poverty level, the prevalence of coronary heart disease, and SNAP acceptance rate.
4. Meaghan - maps, compiled notebook